In [2]:
!conda install -y altair

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.13.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /Users/delphine/opt/miniconda3/envs/jup

  added / updated specs:
    - altair


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.2.2               |     pyhd8ed1ab_0         544 KB  conda-forge
    ca-certificates-2022.12.7  |       h4653dfc_0         142 KB  conda-forge
    certifi-2022.12.7          |     pyhd8ed1ab_0         147 KB  conda-forge
    openssl-3.0.7              |       h03a7124_2         2.0 MB  conda-forge
    toolz-0.12.0               |     pyhd8ed1ab_0          48 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.9 MB

The followi

In [3]:
import pandas as pd
import altair as alt
import numpy as np

In [4]:
# Color mapping for different modes
modes_linelle=['HiFi-only (alt)','HiFi-only (pri)','HiC hap1','HiC hap2','Trio hap1 (pat)','Trio hap2 (mat)']
colors_linelle=['#a6cee3','#1f78b4','#b2df8a', '#33a02c', '#fb9a99','#e31a1c']

modes_byung=['HiFi-only','HiFi-Hic','HiFi-trio','CLR']
colors_byung=['#1f78b4', '#33a02c', '#e31a1c','#ff7f00']

## Linelle's graphs

In [871]:
def hapmer_graph(df):
    graph = alt.Chart(df,title='A').mark_point(clip=True).encode(
        x = alt.X(
            'pat_hapmer:Q',
            scale=alt.Scale(
                type='log',
                domain=(10,2000000)
            ),
            axis=alt.Axis(
                values=[10,100,1000,10000,1000000]
            ),
            title="Paternal hapmers"
        ),
        y = alt.Y(
            'mat_hapmer:Q',
            scale=alt.Scale(
                type='log',
                domain=(10,2000000)
            ),
            axis=alt.Axis(
                values=[10,100,1000,10000,1000000]
            ),
            title="Maternal hapmers"
        ),
        color=alt.Color(
                    'Assembly',
                    scale=alt.Scale(
                    domain=modes_linelle,
                range=colors_linelle
                    )
                ),    
        size=alt.Size('Size:Q',scale=alt.Scale(range=[10, 400],domain=(10,20000000)))
    ).properties(
        height=400,
        width=400
    )
    return(graph)


#.configure_axis(
#        labelFont='Arial',
#        labelFontSize = 30,
#        titleFont='Arial',
#        titleFontSize=30
#    ).configure_legend(
#        orient='top-left',
#        fillColor='white',
#        labelFont='Arial',
#        labelFontSize=23,
#        title=None,
#        symbolSize=200,
#        symbolStrokeWidth=5,

In [872]:
blob = pd.read_csv('hicsolotrio.allcounts.10x.count',sep='\t')

In [873]:
blob.head()

,Assembly,Contig,pat_hapmer,mat_hapmer,Size
0,bTaeGut2.trim.HiC.hic.hap1.p_ctg,h1tg000001l,5,1012249,11321021
1,bTaeGut2.trim.HiC.hic.hap1.p_ctg,h1tg000002l,467388,203,14597216
2,bTaeGut2.trim.HiC.hic.hap1.p_ctg,h1tg000003l,20,654014,4314514
3,bTaeGut2.trim.HiC.hic.hap1.p_ctg,h1tg000004l,755214,172,5769299
4,bTaeGut2.trim.HiC.hic.hap1.p_ctg,h1tg000005l,849658,872,18748363


In [874]:
name_map = {}
name_map['bTaeGut2.solo.p2']='HiFi-only (alt)'
name_map['bTaeGut2.solo.p1']='HiFi-only (pri)'
name_map['bTaeGut2.trim.HiC.hic.hap1.p_ctg']='HiC hap1'
name_map['bTaeGut2.trim.HiC.hic.hap2.p_ctg']='HiC hap2'
name_map['bTaeGut2.trio.cutadapt.20211115.dip.hap1.p_ctg']='Trio hap1 (pat)'
name_map['bTaeGut2.trio.cutadapt.20211115.dip.hap2.p_ctg']='Trio hap2 (mat)'

In [875]:
for key in name_map.keys():
    blob.loc[ (blob['Assembly'] == key,'Assembly') ] =  name_map[key]

In [876]:
blob = blob[blob['pat_hapmer'] > 50]
blob = blob[blob['mat_hapmer'] > 50]
blob = blob[blob['mat_hapmer']/(blob['mat_hapmer']+blob['pat_hapmer'])<=.95]
blob = blob[blob['pat_hapmer']/(blob['mat_hapmer']+blob['pat_hapmer'])<=.95]

In [877]:
panel_a = hapmer_graph(blob)
panel_a

alt.Chart(...)

In [878]:
all_2copy = pd.read_csv('all_2copy.tsv',sep='\t')

In [879]:
all_2copy.head()

,Copies,kmer_multiplicity,Count,Assembly,ordering
1,2,20,19707,Trio hap1 (pat),1
2,2,21,20797,Trio hap1 (pat),1
3,2,22,21937,Trio hap1 (pat),1
4,2,23,22992,Trio hap1 (pat),1
5,2,24,24552,Trio hap1 (pat),1


In [880]:
def line_plot(df,cats,cols,title):
    return(
        alt.Chart(df,title=title).mark_line(clip=True,strokeWidth=5,opacity=.75).encode(
            y = alt.Y(
                'Count:Q',
                scale=alt.Scale(domain=(0,1500000)),
                axis=alt.Axis(values=[0,500000,1000000,1500000]),
                title='Count'
            ),
            x = alt.X(
                'kmer_multiplicity:Q',
                scale=alt.Scale(domain=(49,91)),
                axis=alt.Axis(values=[50,60,70,80,90]),
                title='k-mer multiplicity'
            ),
            color=alt.Color(
                'Assembly',
                scale=alt.Scale(
                    #scheme='paired',
                    domain=cats,
                    range=cols
                )
            )
        ).properties(
            height=400,
            width=400
        )
    )

In [881]:
panel_b = line_plot(all_2copy,modes_linelle,colors_linelle,'B')
panel_b

alt.Chart(...)

In [882]:
all_readonly = pd.read_csv('all_readonly.tsv',sep='\t')

In [883]:
all_readonly.head()

,Copies,kmer_multiplicity,Count,Assembly,ordering
1,read-only,20,4929927,HiC hap1,NaN
2,read-only,21,5330840,HiC hap1,NaN
3,read-only,22,5753258,HiC hap1,NaN
4,read-only,23,6179953,HiC hap1,NaN
5,read-only,24,6603988,HiC hap1,NaN


In [884]:
panel_c = line_plot(all_readonly,modes_linelle,colors_linelle,'C')
panel_c

alt.Chart(...)

In [885]:
alt.hconcat(panel_a,panel_b,panel_c,center=True).configure_axis(
            labelFont='Arial',
            labelFontSize = 16,
            titleFont='Arial',
            titleFontSize=16
        ).configure_legend(
            direction='vertical',
            orient='top-left',
            fillColor='white',
            labelFont='Arial',
            labelFontSize=16,
            title=None,
            symbolSize=50,
            symbolStrokeWidth=2,
        ).configure_title(
            fontSize=20,
            font='Arial',
            anchor='start',
            color='black',
            dx=0,
            #dy=42,
        )

alt.HConcatChart(...)

## ByungJuneKo figures

In [886]:
dup = pd.read_csv('Kmer_Dupl.txt',sep='\t',names=['mode','prct'])

In [887]:
dup

,mode,prct
0,CLR,1.872370
1,HiFi-only,1.596290
2,HiFi-Hic,0.892503
3,HiFi-trio,0.726224


In [888]:
panel_B_a = alt.Chart(dup).mark_bar(color='green',opacity=.7).encode(
    x=alt.X('mode',sort=["CLR", "HiFi-only", "HiFi-Hic", "HiFi-trio"], title=None),
    y=alt.Y('prct',title="Proportion of k-mers (%)"),
    color=alt.Color(
                'mode',
                scale=alt.Scale(
                    domain=modes_byung,
                    range=colors_byung
                ),
            legend=None
        
            ),    
    
).properties(
    width=450,
    height=400,
    title = alt.TitleParams(text = 'D', 
            fontSize=20,
            font='Arial',
            anchor='start',
            color='black',
            dx=0,
            dy=-25,
                           )
)

panel_B_a

alt.Chart(...)

In [889]:
exp_col = pd.read_csv('Kmer_ExpCollap.txt',sep='\t',names=['mode','prct','tp'])

In [890]:
exp_col

,mode,prct,tp
0,CLR,10.86030,Expansion
1,HiFi-only,9.40783,Expansion
2,HiFi-Hic,10.13960,Expansion
3,HiFi-trio,9.55756,Expansion
4,CLR,9.81460,Collapse
5,HiFi-only,6.53933,Collapse
6,HiFi-Hic,6.37083,Collapse
7,HiFi-trio,6.79628,Collapse


In [891]:
panel_B_b = alt.Chart(exp_col,title='E').mark_bar(opacity=.7).encode(
    column=alt.Column('mode:N',header=alt.Header(orient="bottom",labelFontSize=16),title=None,
                      sort=['CLR', 'HiFi-only', 'HiFi-Hic', 'HiFi-trio']),
    x=alt.X('tp:N', title=None),
    y=alt.Y('prct',title="Proportion of k-mers (%)"),
    color=alt.Color(
                'mode', 
                scale=alt.Scale(
                    domain=modes_byung,
                    range=colors_byung,
                ),
        legend=None
            ),
    opacity=alt.Opacity('tp:N',title=None,legend=None),
).properties(
    width=300,
    height=400,
).properties(width=100,
             title = alt.TitleParams(text = 'E', 
            fontSize=20,
            font='Arial',
            anchor='start',
            color='black',
            dx=-1,
            dy=-6,
                                    ))

panel_B_b

alt.Chart(...)

In [892]:
pat = pd.read_csv('Paternal_completeness.txt',sep='\t',names=['tp','k_dup','k_com'])
mat = pd.read_csv('Maternal_completeness.txt',sep='\t',names=['tp','k_dup','k_com'])

In [893]:
pat

,tp,k_dup,k_com
0,Default,0.87,79.3481
1,Rebinned,0.83,80.5694


In [894]:
mat

,tp,k_dup,k_com
0,Default,4.37,77.3701
1,Rebinned,3.71,77.1426


In [895]:
def pat_mat(df,lower,upper,lower2,upper2,title):
    dup_line=alt.Chart(df).mark_line(
        color='red',
        opacity=.7,
        strokeWidth=5,
        point={
            "filled": True,
            "fill": "red",
            "size":100
        }
    ).encode(
        x=alt.X('tp',title=None),
        y=alt.Y('k_dup',scale=alt.Scale(domain=(lower, upper)),title="K-mer duplications (%)",axis=alt.Axis(titleColor='red')),
    )
    col_line=alt.Chart(df).mark_line(
        color='blue',
        opacity=.7,
        strokeWidth=5,
        point={
            "filled": True,
            "fill": "blue",
            "size":100
        }
    ).encode(
        x=alt.X('tp'),
        y=alt.Y(
            'k_com',
            scale=alt.Scale(domain=(lower2, upper2)),
            title="K-mer completeness (%)",
            axis=alt.Axis(titleColor='blue')
            ),
    )
    graph = alt.layer(dup_line, col_line).resolve_scale(
        y = 'independent'
    ).properties(
        width=150,
        height=400,
        title=alt.TitleParams(text =title, 
            fontSize=16,
            font='Arial',
            color='black',
                                    )
    )
    return(graph)

In [896]:
panel_B_c = (pat_mat(pat,0,5,75,85,"Male") | pat_mat(mat,0,5,75,85,"Female")).properties(title = alt.TitleParams(text = 'F', 
            fontSize=20,
            font='Arial',
            anchor='start',
            color='black',
            dx=-45,
            dy=-7,
                                    ))
panel_B_c

alt.HConcatChart(...)

In [897]:
panel_d = pd.read_csv('False_Duplication.txt',sep='\t',names=['mode','prct','mb'])

In [898]:
panel_d['text']=[str(x)+" Mb" for x in panel_d['mb']]

In [899]:
panel_d

,mode,prct,mb,text
0,CLR,1.29,13.8,13.8 Mb
1,HiFi-only,0.60,6.7,6.7 Mb
2,HiFi-Hic,0.21,2.4,2.4 Mb
3,HiFi-trio,0.12,1.3,1.3 Mb


In [900]:
bars = alt.Chart(panel_d).mark_bar(color='green',opacity=.7).encode(
    x=alt.X('mode',title=None,sort=['CLR', 'HiFi-only', 'HiFi-Hic', 'HiFi-trio']),
    y=alt.Y('prct',title="Proportion of false duplications (%)"),
    color=alt.Color(
                'mode', 
                legend=None,
                scale=alt.Scale(
                    domain=modes_byung,
                    range=colors_byung,
                ),
    ),
)
labels = alt.Chart(panel_d).mark_text(color='white',dy=15,fontSize=16
).encode(
    x=alt.X('mode',title="Mode",sort=['CLR', 'HiFi-only', 'HiFi-Hic', 'HiFi-trio']),
    y=alt.Y('prct'),
    text=alt.Text('text'),
)

panel_B_d = (bars + labels).properties(
        width=450,
        height=400,
        title = alt.TitleParams(text = 'G', 
            fontSize=20,
            font='Arial',
            anchor='start',
            color='black',
            dx=0,
            dy=-7,
                               ))

panel_B_d

alt.LayerChart(...)

In [901]:
panel_e_hm = pd.read_csv('False_Loss_HM_HapFiltered_Rotated.txt',sep='\t',names=['x','y','v1','v2','v3','v4','v5'])

In [902]:
panel_e_hm = pd.melt(panel_e_hm,id_vars=['x','y'],value_vars=['v3','v4','v5'])

In [903]:
panel_e_hm['Difference in losses (log)']=np.log(panel_e_hm['value']+0.00000001)

In [904]:
panel_e_hm.head()

,x,y,variable,value,Difference in losses (log)
0,CLR,CLR,v3,0.000000,-18.420681
1,CLR,HiFi-only,v3,0.047499,-3.047051
2,CLR,HiFi-Hic,v3,0.038318,-3.261822
3,CLR,HiFi-trio,v3,0.033233,-3.404205
4,HiFi-only,CLR,v3,0.000175,-8.651759


In [905]:
hm = alt.Chart(panel_e_hm).mark_rect().encode(
    x=alt.X('x:N',sort=["CLR", "HiFi-only", "HiFi-Hic", "HiFi-trio"],title=None),
    y=alt.Y('y:N',sort=["CLR", "HiFi-only", "HiFi-Hic", "HiFi-trio"],title=None),
    color=alt.Color('Difference in losses (log):Q',scale=alt.Scale(scheme='greys'))
).properties(
    height=300,
    width=300
)


hm

alt.Chart(...)

In [906]:
panel_e_top = pd.read_csv('False_Loss_HapFiltered.txt',sep='\t',names=['mode','frac','prct','mb'])

In [907]:
panel_e_top['text']=[str(x)+" Mb" for x in panel_e_top['mb']]

In [908]:
panel_e_top

,mode,frac,prct,mb,text
0,CLR,0.039684,3.968351,42.3,42.3 Mb
1,HiFi-only,0.000580,0.057970,0.7,0.7 Mb
2,HiFi-Hic,0.000148,0.014826,0.2,0.2 Mb
3,HiFi-trio,0.003992,0.399241,4.3,4.3 Mb


In [909]:
bars_e = alt.Chart(panel_e_top).mark_bar(color='green',opacity=.7).encode(
    x=alt.X('mode',sort=["CLR", "HiFi-only", "HiFi-Hic", "HiFi-trio"],title="Mode",axis=None),
    y=alt.Y('prct',scale=alt.Scale(domain=[0, 6]), title=["Proportion of","false losses (%)"]),
     color=alt.Color(
                'mode',
                legend=None,
                scale=alt.Scale(
                    domain=modes_byung,
                    range=colors_byung,
                ),
    ),
)

labels_e = alt.Chart(panel_e_top).mark_text(color='black',align="center", baseline="top",dy=-20,fontSize=16
).encode(
    text='text',
x=alt.X('mode',sort=["CLR", "HiFi-only", "HiFi-Hic", "HiFi-trio"],title="Mode",axis=None),
    y=alt.Y('prct')
)

top_hist = (bars_e + labels_e).properties(
    height=100,
    width=300,
)

In [910]:
top_hist

alt.LayerChart(...)

In [911]:
panel_B_e = (top_hist & hm).properties(title = alt.TitleParams(text = 'H', 
            fontSize=20,
            font='Arial',
            anchor='start',
            color='black',
            dx=-55,
            dy=-2,
                               ))
panel_B_e

alt.VConcatChart(...)

In [912]:
line2=alt.hconcat(panel_B_a,panel_B_b,panel_B_c).configure_axis(
            labelFont='Arial',
            labelFontSize = 12,
            titleFont='Arial',
            titleFontSize=12
        )
line2

alt.HConcatChart(...)

In [913]:
line3=alt.hconcat(panel_B_d,panel_B_e).configure_axis(
            labelFont='Arial',
            labelFontSize = 16,
            titleFont='Arial',
            titleFontSize=16
        ).configure_legend(
        fillColor='white',
        labelFont='Arial',
        labelFontSize=16,
        title=None,
        symbolSize=200,
        symbolStrokeWidth=5,
    )
line3

alt.HConcatChart(...)

In [914]:
alt.hconcat(panel_a,panel_b,panel_c,center=True).configure_axis(
            labelFont='Arial',
            labelFontSize = 16,
            titleFont='Arial',
            titleFontSize=16
        ).configure_legend(
            direction='vertical',
            orient='top-left',
            fillColor='white',
            labelFont='Arial',
            labelFontSize=16,
            title=None,
            symbolSize=50,
            symbolStrokeWidth=2,
        ).configure_title(
            fontSize=20,
            font='Arial',
            anchor='start',
            color='black',
            dx=0,
            #dy=42,
        )

alt.HConcatChart(...)

In [915]:

alt.vconcat(alt.hconcat(panel_B_a,panel_B_b,panel_B_c),alt.hconcat(panel_B_d,panel_B_e)).configure_axis(
            labelFont='Arial',
            labelFontSize = 16,
            titleFont='Arial',
            titleFontSize= 16
        )

alt.VConcatChart(...)

In [927]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [928]:
# Load coverage data for all modes

bg = pd.read_csv('./ITSN/all.bg.gz', sep='\t', names=['chr','s','e','c'])



In [929]:
# Load gene position in coverage plots for each mode
bounds = pd.read_csv('ITSN/ITSN_Homologs.bed',names=['chr','s','e','c','mode','rank','color'])
# Compute midpoint for text alignment

bounds['mp']=((bounds['e']-bounds['s'])/2)+bounds['s']
bounds['mp'] = bounds['mp'].astype(int)
# Split main dataframe intoi individual dataframes for each contig/mode
# Average coverage using non-overlapping 100bp windows to make dataframes smaller
# Compute window midpoint for graphing 

dfs = {}
for contig in bg['chr'].unique():
    df = bg[bg['chr']==contig].sort_values(by=['s'],ignore_index=True)
    df = df.groupby(df.index // 100).agg({'s':'min','e':'max','c':'mean'})
    df['mp']=((df['e']-df['s'])/2) + df['s']
    df['mp'] = df['mp'].astype(int)
    dfs[contig]=df

In [953]:
# Function for generating individual graph facets

def generate_graph(dataframe_dict,gene_bounds,contig_name):
    source_coverage = dataframe_dict[contig_name]
    title=gene_bounds[gene_bounds['chr']==contig_name]['mode'].values[0]
    color=gene_bounds[gene_bounds['chr']==contig_name]['color'].values[0]
    rank=gene_bounds[gene_bounds['chr']==contig_name]['rank'].values[0]
    
    cvrg = alt.Chart(
        source_coverage,
        title=title
    ).mark_line(
        color=color,
        clip=True,
        strokeWidth=2,
        opacity=.3
    ).encode(
        x=alt.X(
            'mp:Q',
            scale=alt.Scale(
                domain=(
                    0,
                    source_coverage['e'].max()-50000
                )
            ),
            axis=alt.Axis(
                values=[0,200000,400000,600000]
            ),
            title=None
        ),
        y=alt.Y(
            'c:Q',
            title=None,
            axis=alt.Axis(
                values=[0,100,200]
            ),
        ),
    ).properties(
        height=60,
        width=400
    )
    source_genes = gene_bounds[gene_bounds['chr']==contig_name]
    band = alt.Chart(source_genes).mark_rect(
        color='grey',
        opacity=.3
    ).encode(
        x='s',
        x2='e'
    )
    
    mc = str(int(source_coverage[ (source_coverage['s'] > source_genes['s'].min()) & (source_coverage['e'] < source_genes['e'].max())]['c'].mean()))
    
    text = band.mark_text(
        angle=90,
        fontSize=16
    ).encode(
        x = 'mp',
        text = alt.value(mc)
    )
    
    return(cvrg + cvrg.transform_loess('mp','c').mark_line(size=10,clip=True,color=color) + band + text)

In [954]:
# Create graph for all modes

graphs = []
for contig in bounds.sort_values(by=['rank'])['chr'].unique():
    graphs.append(generate_graph(dfs,bounds,contig))

In [955]:
# Render the graph

alt.vconcat(*graphs,spacing=0).resolve_scale(x='shared').configure_axis(
    labelFont='Arial',
    labelFontSize = 16,
    grid=False
).configure_title(
    fontSize=16,
    font='Arial',
    anchor='end',
    color='black',
    dx=0,
    dy=76,
).properties(title = alt.TitleParams(text = 'I', 
            fontSize=20,
            font='Arial',
            anchor='start',
            color='black',
            dx=0,
            dy=0,
                               ))

alt.VConcatChart(...)